In [1]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
! pip install -U git+https://github.com/huggingface/evaluate.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-_jyjngeq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-_jyjngeq
  Resolved https://github.com/huggingface/transformers.git to commit 53194991e9f1d25a86d66f36f2774cf462391f13
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7235963 sha256=9b615d9f5b5de0c0c8dea9f91dcc5a4798e830f711a44a8ba2bcf16945899666
  Stored in directory: /tmp/pip-ephem-wheel-cache-7vj_sfuy/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.2

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from datasets import Dataset

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
df = pd.read_csv('/kaggle/input/text-authorship/prepared_df.csv')

In [4]:
df.head()

,Unnamed: 0,author,book,text,text_w_no_ref
0,0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Аркадий Тимофеевич Аверченко\nСобрание сочинен...,deleted deleted deleted Собрание сочинений в ш...
1,1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Что ж, доля истины здесь была. Никуда не денеш...","Что ж , доля истины здесь была . Никуда не ден..."
2,2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Стремясь вырваться из этой беспросветности, он...","Стремясь вырваться из этой беспросветности , о..."
3,3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Начав сотрудничать в отживавшей свой век «Стре...,Начав сотрудничать в отживавшей свой век « Стр...
4,4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Хотим и название уничтожить, — сказал Аверче...","— Хотим и название уничтожить , — сказал delet..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33449 entries, 0 to 33448
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     33449 non-null  int64 
 1   author         33449 non-null  object
 2   book           33449 non-null  object
 3   text           33449 non-null  object
 4   text_w_no_ref  33449 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [6]:
import numpy as np
from typing import Tuple, Any, Sequence, Optional, Dict, List, Generator, Union

def _split_books_to_target(
        books: pd.DataFrame,
        target: float = 0.5
        ) -> Tuple[int, int, int]:
    label_train_count = 0
    label_total_count = int(np.sum(books["counts"]))
    i = 0
    for _, row in books.iloc[:-1].iterrows():
        num_segments = row.counts
        if not label_train_count:
            i += 1
            label_train_count += num_segments
            continue
        new_train_count = label_train_count + num_segments
        new_share = new_train_count / label_total_count
        old_share = label_train_count / label_total_count
        if abs(target - new_share) > abs(target - old_share):
            break
        i += 1
        label_train_count += num_segments
    return i, label_train_count, label_total_count


def _shuffle_books(
        df: pd.DataFrame,
        author: str,
        random_gen: np.random.Generator,
        cross_val: bool = True
        ) -> pd.DataFrame:
    df_slice = df[df.author == author][["book", "counts"]]
    unique_books = df_slice.drop_duplicates("book")
    if len(unique_books) < 2 and not cross_val:
        raise ValueError(f"too few books of author: {author}")
    permunation = random_gen.permutation(len(unique_books))
    shuffled_books = unique_books.iloc[permunation]
    return shuffled_books


def train_test_split(
        df: pd.DataFrame,
        share: float = 0.5,
        seed: int = 10,
        cross_val: bool = False
        ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """
        разделяет данные на обучающую и тестовую выборки по книгам
    :param df: датафрейм для анализа
    :param share: процент данных для трейна ( тест, соответственно 1-share)
    :param seed: сид для случайных перемешиваний
    :param cross_val: флаг для использования в кросс-валидации
        (более жесткие требования к делению)
    :return:
    """
    rg = np.random.default_rng(seed)
    labels = df.author.unique()
    shuffled_labels = labels[rg.permutation(len(labels))]

    train_label = []
    test_label = []
    train_counts = 0
    test_counts = 0

    for label in shuffled_labels:
        shuffled_books = _shuffle_books(
            df=df,
            author=label,
            random_gen=rg,
            cross_val=cross_val)
        split_idx, segm_train, total_semg = _split_books_to_target(
            shuffled_books,
            share)

        train_label.extend(shuffled_books.iloc[:split_idx].book)
        test_label.extend(shuffled_books.iloc[split_idx:].book)
        train_counts += segm_train
        test_counts += total_semg - segm_train

    train_df = df[df["book"].isin(set(train_label))]
    test_df = df[df["book"].isin(set(test_label))]
    assert len(train_df) + len(test_df) == len(df), \
        "split is wrong"
    y_train = train_df["author"]
    y_test = test_df["author"]
    return train_df, test_df, y_train, y_test

In [7]:
from sklearn.preprocessing import LabelEncoder

target_encoder = LabelEncoder()
df['label'] = target_encoder.fit_transform(df.author)
book_counts = df.groupby('book').size()
df['counts'] = df.book.map(book_counts)

In [8]:
NUM_LABELS = df.label.nunique()
config = BertConfig.from_pretrained('sberbank-ai/ruBert-base')
config.num_labels = NUM_LABELS
model=BertForSequenceClassification.from_pretrained('sberbank-ai/ruBert-base', config=config)
tokenizer=BertTokenizer.from_pretrained('sberbank-ai/ruBert-base', do_lower_case=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
dataset = Dataset.from_dict({'text': df.text_w_no_ref.tolist(), 
                                  'label': df.label.tolist()})

def tokenize_func(sample):
    return tokenizer(sample['text'], 
                    truncation=True, 
                    max_length=512)

dataset = (dataset.map(tokenize_func)
                .remove_columns('text')
                .with_format('torch'))

  0%|          | 0/33449 [00:00<?, ?ex/s]

In [10]:
class MaskingDataset:
    def __init__(self, dataset, tokenizer, p=0.2):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.p = p
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        input_ids = item['input_ids']
        attention_mask = item['attention_mask']
        mask = torch.bernoulli(torch.full_like(input_ids, self.p, dtype=torch.float32)).to(bool)
        input_ids[mask] = self.tokenizer.mask_token_id
        attention_mask[mask] = 0
        return item

In [11]:
weights = torch.from_numpy(df.shape[0] /
                           NUM_LABELS /
                           df.label.value_counts().sort_index().values).float()
weights

tensor([ 0.7399,  0.7041,  0.8762,  2.5960,  0.6629,  0.3177, 31.8562,  0.8290,
         7.2400,  1.5507,  2.1629,  0.8466,  0.5390,  3.5284,  4.1219])

In [12]:
from transformers import Trainer
from torch import nn

DEFAULT_DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class MyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=weights).to(DEFAULT_DEVICE)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [13]:
def get_group(model, key, eps):
    if key == 'embedding':
        names = ['bert.embeddings.']
        coef = eps ** 12
    elif key == 'classifier':
        names = ['bert.pooler.', 'classifier.']
        coef = eps
    else:
        names = [f'bert.encoder.layer.{key}.']
        coef = eps ** (11 - key)
    return [p for k, p in model.named_parameters() if any(k.startswith(name) for name in names)], coef

In [14]:
from transformers import AdamW

def get_optimizer(model, learning_rate=5.0e-05, mode='normal', layer_decay=0.95):
    if mode == 'layer_decay':
        param_keys = ['embedding', 'classifier'] + list(range(12))
        param_groups = []
        for key in param_keys:
            params, coef = get_group(model, key, layer_decay)
            param_groups.append({'params': params, 'lr': learning_rate * coef})
        optimizer = AdamW(param_groups, lr=learning_rate, eps=1e-08)
        return optimizer
    if mode == 'freeze_attention':
        for k, p in model.named_parameters():
            if k.find('attention.self') != -1:
                p.requires_grad = False
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-08)
    return optimizer

optimizer = get_optimizer(model, mode='layer_decay', layer_decay=0.9)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='rubert_text_authorship',
    gradient_accumulation_steps=4,
    per_device_train_batch_size=8,
    num_train_epochs=4.0,
    save_strategy='steps',
    logging_steps=100,
    save_steps=100,
    save_total_limit=2,
    report_to='wandb'
)

In [16]:
import evaluate

f1, precision, recall = (evaluate.load(metric) for metric in ['f1', 'precision', 'recall'])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    output = {}
    for metric in [f1, precision, recall]:
        output.update(metric.compute(predictions=predictions,
                                    references=labels,
                                    average='macro'))
    return output

In [17]:
import wandb
wandb.login(key='3414b6a58a2f4bdbcf311e8fbdd360736f048424')

trainer = MyTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    optimizers=(optimizer, None)
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [18]:
trainer.train()

wandb: Currently logged in as: vladimir-golod22 (vg_dl). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230627_184538-24ce39b8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run effortless-energy-27
wandb: ⭐️ View project at https://wandb.ai/vg_dl/huggingface
wandb: 🚀 View run at https://wandb.ai/vg_dl/huggingface/runs/24ce39b8
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,1.967000
200,0.983800
300,0.634900
400,0.440300
500,0.316800
600,0.198500
700,0.162400
800,0.158100
900,0.114900
1000,0.111500


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=2088, training_loss=0.2587170282885489, metrics={'train_runtime': 8381.7316, 'train_samples_per_second': 15.963, 'train_steps_per_second': 0.249, 'total_flos': 3.502778727305088e+16, 'train_loss': 0.2587170282885489, 'epoch': 3.99})

In [19]:
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                    train/epoch ▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
wandb:              train/global_step ▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
wandb:            train/learning_rate ██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
wandb:                     train/loss █▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                    train/epoch 3.99
wandb:              train/global_step 2088
wandb:            train/learning_rate 0.0
wandb:                     train/loss 0.0076
wandb:               train/total_flos 3.502778727305088e+16
wandb:               train/train_loss 0.25872
wandb:            train/train_runtime 8381.7316
wandb: train/train_samples_per_second 15.963
wandb:   train/train_steps_per_second 0.249
wandb: 
wandb: 🚀 View run effortles

In [20]:
trainer.save_model()